In [60]:
from neo4j import GraphDatabase

In [61]:
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "qwer123456")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver

In [62]:
driver.execute_query("match (n) detach delete n", database_='neo4j')  # 删除所有节点和关系

cql_string = """CREATE
(xiaowang_:Person {name: 'xiaowang', bornIn: '湖南', age:34, email: 'aaa.com', love: ['eat', 'play', 'study']}),
(dc_:Person {name: 'dc', bornIn: '广东', age:41, email: 'bbb.com', love: ['sleep', 'run']}),
(xiaohong_:Person {name: 'xiaohong', bornIn: '上海', age:51, email: 'ccc.com', love: ['eat', 'run', 'study']}),
(xiaohei_:Person {name: 'xiaohei', bornIn: '湖南', age:66, love: ['eat', 'play', 'read']}),
(dmm_:Person {name: 'dmm', bornIn: '湖南', age:8, love: ['eat', 'play', 'swim']}),
(tom_:Person {name: 'tom', bornIn: '美国', age:38, love: ['eat', 'swim', 'driver']}),

(dhxy:Movie {title: '大话西游'}),
(bfx:Movie {title: '蝙蝠侠'}),
(hkdg:Movie {title: '黑客帝国'}),

(xiaowang_)-[:like{time:15}]->(dhxy),
(dc_)-[:dislike{time:52}]->(bfx),
(xiaohong_)-[:like{time:45}]->(hkdg),
(xiaohei_)-[:like{time:5}]->(hkdg),
(dmm_)-[:like]->(hkdg),
(dmm_)-[:dislike]->(bfx)"""
driver.execute_query(cql_string, database_='neo4j')

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x00000295AE0DE680>, keys=[])

In [63]:

# 比较运算符:> < = >= <= <>(不等于)
r0 = driver.execute_query("match (p:Movie) "
                     "where p.title='蝙蝠侠' "
                     "return p").records
for i in r0:
    print(i.data())

{'p': {'title': '蝙蝠侠'}}


In [64]:
r0 = driver.execute_query("match (n:Person)-[k:like]->(f) "
                          "where k.time < 21 "
                          "return f.title").records
for i in r0:
    print(i.data())

{'f.title': '大话西游'}
{'f.title': '黑客帝国'}


In [65]:
# 布尔运算符:or and not
r0 = driver.execute_query("match (p:Person) "
                     "where p.name='dmm' or p.age > 60 "
                     "return p").records
for i in r0:
    print(i.data())

{'p': {'love': ['eat', 'play', 'read'], 'bornIn': '湖南', 'name': 'xiaohei', 'age': 66}}
{'p': {'love': ['eat', 'play', 'swim'], 'bornIn': '湖南', 'name': 'dmm', 'age': 8}}


In [66]:
# 字符串正则表达式匹配(=~)
# .:任意字符
# *:0次或多次
r0 = driver.execute_query("match (p:Person) "
                          "where p.name=~'d.*' "
                          "return p").records
for i in r0:
    print(i.data())

{'p': {'love': ['sleep', 'run'], 'bornIn': '广东', 'name': 'dc', 'age': 41, 'email': 'bbb.com'}}
{'p': {'love': ['eat', 'play', 'swim'], 'bornIn': '湖南', 'name': 'dmm', 'age': 8}}


In [67]:
# 匹配字符\:\\
# 匹配字符.:\\.
# 匹配字符*:\\*
r0 = driver.execute_query("match (n:Person) "
                          "where n.email =~ '.*\\.com' "
                          "return n.name").records
for i in r0:
    print(i.data())

{'n.name': 'xiaowang'}
{'n.name': 'dc'}
{'n.name': 'xiaohong'}


In [68]:
# 检查列表中是否存在某个元素:in
r0 = driver.execute_query("match (a:Person) "
                          "where 'eat' in a.love "
                          "return a.name, a.love").records
for i in r0:
    print(i.data())

{'a.name': 'xiaowang', 'a.love': ['eat', 'play', 'study']}
{'a.name': 'xiaohong', 'a.love': ['eat', 'run', 'study']}
{'a.name': 'xiaohei', 'a.love': ['eat', 'play', 'read']}
{'a.name': 'dmm', 'a.love': ['eat', 'play', 'swim']}
{'a.name': 'tom', 'a.love': ['eat', 'swim', 'driver']}


In [69]:
driver.close()